In [1]:
def h(x):
    return(x[0]*x[1] + x[2]*x[3] + x[4]*x[5] + x[6]*x[7] + x[0]*x[4]*x[8] + l0 + l0)

def lfsr_update_fw(S):
    temp = S[0] + S[7] + S[38] + S[70] + S[81] + S[96] + l0 + l0
    S = S[1:128] + [temp]
    return S

def lfsr_update_bk(S):
    temp = S[127] + S[6] + S[37] + S[69] + S[80] + S[95] + l0 + l0
    S = [temp] + S[0:127]
    return S

def nfsr_update_fw(B,s):
    temp = s + B[0] + B[26] + B[56] + B[91] + B[96] + B[3]*B[67] + B[11]*B[13] + B[17]*B[18]\
    + B[27]*B[59] + B[40]*B[48] + B[61]*B[65] + B[68]*B[84] + B[88]*B[92]*B[93]*B[95] \
    + B[22]*B[24]*B[25] + B[70]*B[78]*B[82] + l0 + l0
    B = B[1:128] + [temp]
    return B

def nfsr_update_bk(B,s):
    temp = s + B[127] + B[25] + B[55] + B[90] + B[95] + B[2]*B[66] + B[10]*B[12] + B[16]*B[17]\
    + B[26]*B[58] + B[39]*B[47] + B[60]*B[64] + B[67]*B[83] + B[87]*B[91]*B[92]*B[94] \
    + B[21]*B[23]*B[24] + B[69]*B[77]*B[81] + l0 + l0
    B = [temp] + B[0:127]
    return B
    
def keystream(S,B):
    z = h([B[12], S[8], S[13], S[20], B[95], S[42], S[60], S[79], S[94]]) + S[93] + B[2] \
    + B[15] + B[36] + B[45] + B[64] + B[73] + B[89] + l0 + l0
    return z
    
def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp

In [3]:
import time
import csv 
import numpy as np
import gurobipy as gp
from gurobipy import GRB


start2 = time.time()

filename = 'Grain-128a_TMDTO_bkd.txt'
csv_file = 'Grain128a_TMDTO_MILP_DATA_bkd.csv'

keys = 35
keys_b = 0
keys_f = keys-keys_b


pattern = [0]
for i in range(1,max(keys_f,keys_b)+1,1):
    if i < keys_b+1:
        pattern = pattern + [-i]
    if i < keys_f:
        pattern = pattern + [i]
#pattern = [-i for i in range(1,keys_b+1,1)] + [i for i in range(keys_f)]
#pattern = [i for i in range(-keys_b,0,1)] + [i for i in range(keys_f)]



print("Pattern of the keystream bits ", pattern)
state_len = 256
r = [128,128]
V = BooleanPolynomialRing(state_len,['l%d'%i for i in range(r[0])] + ['n%d'%i for i in range(r[1])])
V.inject_variables()
g = list(V.gens())

S = g[:r[0]]
B = g[r[0]:]



R = [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126 , 127 , 128+3, 128+21, 128+23, 128+24, 128+25, 128+27, 128+28, 128+33, 128+34, 128+35, 128+36, 128+38, 128+39, 128+41, 128+43, 128+56, 128+59, 128+61, 128+62, 128+88, 128+93]
F = [128 + 32, 128 + 37, 128 + 40, 128 + 42, 128 + 44, 128 + 45, 128 + 116, 128 + 117, 128 + 118, 128 + 119, 128 + 121, 128 + 122, 128 + 124, 128 + 126]
G = [i for i in range(state_len) if i not in R+F]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


linear_b = [0]*keys_b
linear_f = [0]*keys_f
n_mon = 0
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])

    Z_b[loop] = keystream(S,B)
    monom = list(Z_b[loop].monomials())
    linear_b[loop] = [m for m in monom if m.degree() == 1]
    for m in monom:
        if m.degree()>1:
            n_mon += 1
    #print("backward loop 1st part ", loop)
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


for loop in range(keys_f):
    Z_f[loop] = keystream(S,B)
    monom = list(Z_f[loop].monomials())
    linear_f[loop] = [m for m in monom if m.degree() == 1]
    for m in monom:
        if m.degree()>1:
            n_mon += 1
    
    #print("forward loop 1st part ",loop)
    if loop != keys_f-1:
        B=nfsr_update_fw(B,S[0])
        S=lfsr_update_fw(S) 
#print("1st part done")


index = 0
Z = [0]*(keys_f+keys_b)
linear = [0]*(keys_f + keys_b)
linear_new = [0]*(keys_f + keys_b)
linear_all = []

for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
        linear[index] = linear_b[-i-1]
    if i >= 0:
        Z[index] = Z_f[i]
        linear[index] = linear_f[i]
    index += 1

for loop in range(len(Z)):
    lin = []
    for i in linear[loop]:
        if g.index(i) not in linear_all:
            lin = lin + [g.index(i)]
            
    linear_new[loop] = lin
    linear_all = linear_all + lin
    
#########Gurobi Modelling Starts###############
model = gp.Model()


v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
aux = [i for i in range(n_mon)]

#Let D, F, G represents the class of determined/recovery, Fixed bit, and Guess bit
#Since these are set and we need variables to denote R,F and G therefore let 2=R, 3=F and 4=G
#x represents variables for each register and m contains 3 variables for each x[i] i.e. for R,F and G
x = model.addVars(len(S)+len(B), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
m = model.addVars(len(S)+len(B), len(v), vtype = GRB.BINARY, name = "m")
q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")
#p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
#r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
model.update()

model.addConstrs((m[i,0] == 1) for i in R)
model.addConstrs((m[i,1] == 1) for i in F)
model.addConstrs((m[i,2] == 1) for i in G)

#Objective function is to maximize recovery bit and minimize the number of FIXED bit
model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)


model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))

#Number of recovered bit should be less than or equal to the number of keystream equations
#model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) == keys)
model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)

count = 0
cnt = 0
for loop in range(keys_f + keys_b):
    z = Z[loop]
    #z_var contains the index of variables in Z
    #z_var = np.array(z.variables())
    #z_var = [g.index(i) for i in z_var]
    z_monom = np.array(z.monomials())

    z_linear = [g.index(i) for i in z_monom if i.degree() == 1]

    #z_nlinear = [list(i.variables()) for i in z_monom if len(i.variables()) > 1]
    #z_nlinear = [g.index(z_nlinear[i][j]) for i in range(len(z_nlinear)) for j in range(len(z_nlinear[i]))]
    #z_nlinear = list(set(z_nlinear))


    #linear_new = [i for i in z_linear if i not in (linear + z_nlinear)]
    #z_linear_org = [i for i in z_linear if i not in z_nlinear]


    #One of the linear term variables which is not in the previous linear term has one recovered bit
    model.addConstr(gp.quicksum(m[i,0] for i in linear_new[loop]) <= 1)    
    #model.addConstr(gp.quicksum(m[i,0] for i in z_linear) <= loop + 1) 



    for mon in z_monom:
        if mon.degree() > 1:
            var = list(mon.variables())
            var = [g.index(i) for i in var]
            inters = [0]*len(linear_new)
            n = [0]*len(linear_new)
            for l in range(len(linear_new)):
                inters[l] = intersection(var,linear_new[l])
                n[l] = len(inters[l])

            C_prev = [n[i] for i in range(loop)] 
            C_current = [n[loop]]
            C_future = [n[i] for i in range(loop+1,keys,1)]

            #Case 1: recovered var in mon and is in linear
            #if sum(C_prev) > 0 and sum(C_current) == 0 and sum(C_future) == 0 :
            #    #if recovery found then guess
            #    model.addConstr((p[count] == 0) >> (gp.quicksum(m[i,0] for i in var) == 0))
            #    model.addConstr((p[count] == 1) >> (gp.quicksum((m[i,1] + m[i,2]) for i in var) >= len(var) - sum(C_prev)))
            #    count += 1

            #Case 2: recovered var in mon and is in z_linear-linear
            if (sum(C_current + C_future) > 0):
                #if recovery found then fix to zero
                prev = []
                for k in range(loop):
                    prev = prev + inters[k]
                #var_ge = [i for i in var if i not in prev]
                
                var_ge = []
                for k in range(loop,len(linear_new),1):
                    var_ge = var_ge + inters[k]

                model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_ge) == 0))
                model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                count += 1

            #if (sum(C_prev + C_current + C_future) == 0): 
            #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
    print("2nd part loop ", loop)
print("2nd part done")
import time

model.params.outputflag = 0
start = time.time()
model.optimize()
end = time.time() - start

print("Keystream = ",keys)
print("status",model.STATUS)

f = open(filename,'a')
f.write("\nKeystream = " + str(keys))
f.write("\nStatus " + str(model.STATUS) + "\n")

if model.STATUS == 2:
    val = model.x
    R = []
    F = []
    G = []
    for i in range(state_len):
        if round(val[i]) == 2:
            R.append(i)
        if round(val[i]) == 3:
            F.append(i)
        if round(val[i]) == 4:
            G.append(i)



    Rec = [g[i] for i in R]
    Fix = [g[i] for i in F]
    Guess = [g[i] for i in G]
    print("Recovery Bit ",Rec)
    print("Fixed Bit ",Fix)
    print("Guessed Bit ",Guess)
    
    f.write(str("Recovery Bit ") + str(Rec) + "\n")
    f.write(str("Fixed Bit ") + str(Fix) + "\n")
    f.write(str("Guessed Bit ") + str(Guess) + "\n")
    
    print(len(R),len(F))
    if (len(R) + len(F) + len(G) == state_len) and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
        print("Length verified")
        f.write("\nLength Verified\n\n")
end2 = time.time()-start2
print("Gurobi Time = ", end)
print("Overall Time = ", end2)
f.close()



Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)


#Assigning variables to the registers
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0

        
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])

    Z_b[loop] = keystream(S,B)
    
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


for loop in range(keys_f):
    Z_f[loop] = keystream(S,B)
    
    if loop != keys_f-1:
        B=nfsr_update_fw(B,S[0])
        S=lfsr_update_fw(S) 


index = 0
Z = [0]*(keys_f+keys_b)

for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
    if i >= 0:
        Z[index] = Z_f[i]
    index += 1


R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys_f+keys_b):
    z = Z[i]
    monom = list(z.monomials())
    t_var = list(z.variables())

    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            f = open(filename,'a')
            f.write("\nError --- Future recovery bits found\n" + str(m))
            f.close()
            print("Error --- Future recovery bits found")
            print(m)
            flag = 0
            break

    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    #print("Recovery bits till now in linear terms ",L_rec)

f = open(filename,'a')
if flag == 0:
    print("Failed")
    f.write("\nFailed\n")
else:
    print("Passed")
    f.write("\nPassed\n")
    with open(csv_file,'a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([keys_f,keys_b,len(R),len(F),Rec,Fix,pattern])
f.close()


Pattern of the keystream bits  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
Defining l0, l1, l2, l3, l4, l5, l6, l7, l8, l9, l10, l11, l12, l13, l14, l15, l16, l17, l18, l19, l20, l21, l22, l23, l24, l25, l26, l27, l28, l29, l30, l31, l32, l33, l34, l35, l36, l37, l38, l39, l40, l41, l42, l43, l44, l45, l46, l47, l48, l49, l50, l51, l52, l53, l54, l55, l56, l57, l58, l59, l60, l61, l62, l63, l64, l65, l66, l67, l68, l69, l70, l71, l72, l73, l74, l75, l76, l77, l78, l79, l80, l81, l82, l83, l84, l85, l86, l87, l88, l89, l90, l91, l92, l93, l94, l95, l96, l97, l98, l99, l100, l101, l102, l103, l104, l105, l106, l107, l108, l109, l110, l111, l112, l113, l114, l115, l116, l117, l118, l119, l120, l121, l122, l123, l124, l125, l126, l127, n0, n1, n2, n3, n4, n5, n6, n7, n8, n9, n10, n11, n12, n13, n14, n15, n16, n17, n18, n19, n20, n21, n22, n23, n24, n25, n26, n27, n28, n29, n30, n31, n32, n33, n34, n35, n

In [ ]:
#Assigning variables to the registers
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0

        
Z_f = [0]*keys_f
Z_b = [0]*keys_b

for loop in range(keys_b):
    S = lfsr_update_bk(S)
    B = nfsr_update_bk(B,S[0])

    Z_b[loop] = keystream(S,B)
    
    
S = g[:r[0]]
B = g[r[0]:]

for i in F:
    if i < r[0]:
        S[i] = 0
    else:
        B[i-r[0]] = 0


for loop in range(keys_f):
    Z_f[loop] = keystream(S,B)
    
    if loop != keys_f-1:
        B=nfsr_update_fw(B,S[0])
        S=lfsr_update_fw(S) 


index = 0
Z = [0]*(keys_f+keys_b)

for i in pattern:
    if i < 0:
        Z[index] = Z_b[-i-1]
    if i >= 0:
        Z[index] = Z_f[i]
    index += 1


R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys_f+keys_b):
    z = Z[i]
    monom = list(z.monomials())
    t_var = list(z.variables())

    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            f = open(filename,'a')
            f.write("\nError --- Future recovery bits found\n" + str(m))
            f.close()
            print("Error --- Future recovery bits found")
            print(m)
            flag = 0
            break

    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    #print("Recovery bits till now in linear terms ",L_rec)

f = open(filename,'a')
if flag == 0:
    print("Failed")
    f.write("\nFailed\n")
else:
    print("Passed")